## The Autoencoder for the multivariate time series

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from kan import KAN, KANLinear

### Loading the data

In [2]:
from sklearn.preprocessing import StandardScaler
# Load and preprocess data from a CSV file
def load_data(file_path):
    # Assuming the CSV file has a header row and each row is a time series sample
    data = pd.read_csv(file_path)
    return data

# Custom PyTorch dataset class for time series data
class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data.values, dtype=torch.float32)

        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [3]:
file_path = '/home/mohammed/moment/data/Repressilator/One_Parameter/Repressilator_training.csv'
data = load_data(file_path)

# Calculate mean and standard deviation for normalization
mean = data.mean()
std = data.std()

# Create the dataset and data loader
dataset = TimeSeriesDataset(data)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True)

### The model

In [4]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.LeakyReLU(),
            nn.Linear(64, latent_dim),
            nn.Tanh()
        #    # add self attention layer
        )

        #self.encoder = KAN([input_dim, 10,10, latent_dim],grid_range=[-4,4],grid_size=10)
        #self.decoder = KAN([latent_dim, 10,10, input_dim],grid_range=[-4,4],grid_size=10)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.LeakyReLU(),
        #    #nn.Linear(32, 64),
        #    #nn.LeakyReLU(),
            nn.Linear(64, input_dim)
        )
        # Parameters for normalization
        self.mean = None
        self.std = None
        
    def normalize(self, x):

        return (x - self.mean) / self.std
    
    def denormalize(self, x):
        # Denormalize the data using precomputed mean and std
        return x * self.std + self.mean
    
    def forward(self, x):
        # Normalize input
        x = self.normalize(x)
        
        # Encode and decode

        encoded = self.encoder(x)
        
        decoded = self.decoder(encoded)

        
        # Denormalize output
        decoded = self.denormalize(decoded)
        return decoded

    def encode(self, x):
        # Normalize input
        x = self.normalize(x)
        
        # Encode

        encoded = self.encoder(x)
        return encoded
    def decode(self, x):
        # Decode
        decoded = self.decoder(x)
        # Denormalize output
        decoded = self.denormalize(decoded)
        return decoded

### training the model

In [5]:
# Hyperparameters
input_dim = data.shape[1]   # Number of features in the multivariate time series
latent_dim = 6              # Target dimension for the univariate representation
num_epochs = 30
learning_rate = 0.001

# Initialize the autoencoder
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = Autoencoder(input_dim=input_dim, latent_dim=latent_dim)

# Set normalization parameters
autoencoder.mean = torch.tensor(mean.values, dtype=torch.float32)
autoencoder.std = torch.tensor(std.values, dtype=torch.float32)

# Check if GPU is available and move the model and tensors to GPU

autoencoder = autoencoder.to(device)
autoencoder.mean = autoencoder.mean.to(device)
autoencoder.std = autoencoder.std.to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)


In [6]:
# Training loop
for epoch in range(num_epochs):
    loss1 = 0
    for batch in data_loader:
        batch = batch.to(device)  # Move data to GPU

        # Forward pass
        reconstructed = autoencoder(batch)

        # Compute loss
        loss = criterion(reconstructed, batch)
        loss1 = loss1 + loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss1 = loss1 / len(data_loader)
    #if (epoch + 1) % 10 == 0:
    #print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss1: {loss1.item():.4f}')

# Save the trained model
torch.save(autoencoder.state_dict(), 'autoencoder_model.pth')

# Optional: Evaluate the model on test data
def evaluate_model(autoencoder, data_loader):
    autoencoder.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)  # Move data to GPU
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            total_loss += loss.item()
    average_loss = total_loss / len(data_loader)
    return average_loss

# Evaluate on training data (replace with test data if available)
train_loss = evaluate_model(autoencoder, data_loader)
print(f'Training Data Loss: {train_loss:.4f}')

KeyboardInterrupt: 

In [7]:
# load model from autoencoder_model.pth

autoencoder.load_state_dict(torch.load('autoencoder_model.pth'))

<All keys matched successfully>

## MOMENT

In [16]:
import momentfm
from momentfm import MOMENTPipeline

model = MOMENTPipeline.from_pretrained(
    "AutonLab/MOMENT-1-large",
    model_kwargs={
        'task_name': 'forecasting',
        'forecast_horizon': 192,
        'head_dropout': 0.1,
        'weight_decay': 0,
        'freeze_encoder': True, # Freeze the patch embedding layer
        'freeze_embedder': True, # Freeze the transformer encoder
        'freeze_head': False, # The linear forecasting head must be trained
    },
)

In [17]:
model.init()
print(model)

MOMENTPipeline(
  (normalizer): RevIN()
  (tokenizer): Patching()
  (patch_embedding): PatchEmbedding(
    (value_embedding): Linear(in_features=8, out_features=1024, bias=False)
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
  

In [18]:
print("Unfrozen parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print('    ', name)

Unfrozen parameters:
     head.linear.weight
     head.linear.bias


In [19]:
import numpy as np
import torch
import os
import torch.cuda.amp
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR
from tqdm import tqdm

from momentfm.utils.utils import control_randomness
#from momentfm.data.informer_dataset import InformerDataset
from momentfm.utils.forecasting_metrics import get_forecasting_metrics




In [23]:

## Iformer dataset using 192 forecasting horizon and 4 timeseries for each 1024 consisting of 512 context and 192 forecasting
from typing import Optional

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


class InformerDataset:
    def __init__(
        self,
        forecast_horizon: Optional[int] = 192,
        data_split: str = "train",
        data_stride_len: int = 80,
        task_name: str = "forecasting",
        random_seed: int = 42,
    ):
        """
        Parameters
        ----------
        forecast_horizon : int
            Length of the prediction sequence.
        data_split : str
            Split of the dataset, 'train' or 'test'.
        data_stride_len : int
            Stride length when generating consecutive
            time series windows.
        task_name : str
            The task that the dataset is used for. One of
            'forecasting', or  'imputation'.
        random_seed : int
            Random seed for reproducibility.
        """

        self.seq_len = 512
        self.forecast_horizon = forecast_horizon
        self.full_file_path_and_name = "/home/mohammed/moment/data/Repressilator/One_Parameter/Repressilator_training.csv"
        self.validation_data ="/home/mohammed/moment/data/Repressilator/One_Parameter/Repressilator_validation.csv"
        self.data_split = data_split
        self.data_stride_len = data_stride_len
        self.task_name = task_name
        self.random_seed = random_seed

        # Read data
        self._read_data()

    def _read_data(self):
        self.scale = StandardScaler()
        if self.data_split == "train":
            df = pd.read_csv(self.full_file_path_and_name)
        elif self.data_split == "test":
            df = pd.read_csv(self.validation_data)
        self.length_timeseries_original = df.shape[0]
        self.n_channels = df.shape[1] - 1

        df = df.infer_objects(copy=False).interpolate(method="cubic")
        self.data = df.values

        self.length_timeseries = self.data.shape[0]

    def __getitem__(self, index):
        x = (index // 4)* 1024
        y = (index % 4)* self.data_stride_len
        seq_start = x+y
        seq_end = seq_start + self.seq_len
        input_mask = np.ones(self.seq_len)

        if self.task_name == "forecasting":
            pred_end = seq_end + self.forecast_horizon

            if pred_end > self.length_timeseries:
                pred_end = self.length_timeseries
                #seq_end = seq_end - self.forecast_horizon
                seq_end = self.length_timeseries - self.forecast_horizon
                seq_start = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T
            forecast = self.data[seq_end:pred_end, :].T

            return timeseries, forecast, input_mask

        elif self.task_name == "imputation":
            if seq_end > self.length_timeseries:
                seq_end = self.length_timeseries
                seq_end = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T

            return timeseries, input_mask

    def __len__(self):
        if self.task_name == "imputation":
            return (self.length_timeseries - self.seq_len) // self.data_stride_len + 1
        elif self.task_name == "forecasting":
            return (
                self.length_timeseries - self.seq_len - self.forecast_horizon
            ) // self.data_stride_len + 1

In [21]:

# This is informer dataset for 512 context and 512 forecasting horizon 
from typing import Optional

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


class InformerDataset:
    def __init__(
        self,
        forecast_horizon: Optional[int] = 192,
        data_split: str = "train",
        data_stride_len: int = 1024,
        task_name: str = "forecasting",
        random_seed: int = 42,
    ):
        """
        Parameters
        ----------
        forecast_horizon : int
            Length of the prediction sequence.
        data_split : str
            Split of the dataset, 'train' or 'test'.
        data_stride_len : int
            Stride length when generating consecutive
            time series windows.
        task_name : str
            The task that the dataset is used for. One of
            'forecasting', or  'imputation'.
        random_seed : int
            Random seed for reproducibility.
        """

        self.seq_len = 512
        self.forecast_horizon = forecast_horizon
        self.full_file_path_and_name = "/home/mohammed/moment/data/Repressilator/One_Parameter/Repressilator_training.csv"
        self.validation_data ="/home/mohammed/moment/data/Repressilator/One_Parameter/Repressilator_validation.csv"
        self.data_split = data_split
        self.data_stride_len = data_stride_len
        self.task_name = task_name
        self.random_seed = random_seed

        # Read data
        self._read_data()

    def _read_data(self):
        self.scaler = StandardScaler()
        if self.data_split == "train":
            df = pd.read_csv(self.full_file_path_and_name)
            print(df.shape)
        elif self.data_split == "test":
            df = pd.read_csv(self.validation_data)
        self.length_timeseries_original = df.shape[0]
        self.n_channels = df.shape[1] - 1

        df = df.infer_objects(copy=False).interpolate(method="cubic")
        self.data = df.values


        #train_data = df
        #self.scaler.fit(train_data.values)
        #df = self.scaler.transform(df.values)
        #self.data = df
        #self.data = df.values

        self.length_timeseries = self.data.shape[0]
        #print(f'the length of the time series is {self.length_timeseries}')

    def __getitem__(self, index):
        x = index * 1024
        #y = index* self.data_stride_len
        #seq_start = x+y
        seq_start = x
        seq_end = seq_start + self.seq_len
        input_mask = np.ones(self.seq_len)

        if self.task_name == "forecasting":
            pred_end = seq_end + self.forecast_horizon

            if pred_end > self.length_timeseries:
                pred_end = self.length_timeseries
                #seq_end = seq_end - self.forecast_horizon
                seq_end = self.length_timeseries - self.forecast_horizon
                seq_start = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T
            forecast = self.data[seq_end:pred_end, :].T

            return timeseries, forecast, input_mask

        elif self.task_name == "imputation":
            if seq_end > self.length_timeseries:
                seq_end = self.length_timeseries
                seq_end = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T

            return timeseries, input_mask

    def __len__(self):
        if self.task_name == "imputation":
            return (self.length_timeseries - self.seq_len) // self.data_stride_len + 1
        elif self.task_name == "forecasting":
            return (
                self.length_timeseries - self.seq_len - self.forecast_horizon
            ) // self.data_stride_len + 1

In [24]:
# Set random seeds for PyTorch, Numpy etc.
control_randomness(seed=13)

# Load data
train_dataset = InformerDataset(data_split="train", random_seed=13, forecast_horizon=192)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#mu = train_dataset.scale.mean_
#std = train_dataset.scale.scale_
# make the mean and the standard deviation to be in device
#mu = torch.tensor(mu, dtype=torch.float32).to(device)
#std = torch.tensor(std, dtype=torch.float32).to(device)

test_dataset = InformerDataset(data_split="test", random_seed=13, forecast_horizon=192)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)



In [25]:

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

autoencoder.to(device)
cur_epoch = 0
max_epoch = 10

# Move the model to the GPU
model = model.to(device)

# Move the loss function to the GPU
criterion = criterion.to(device)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Create a OneCycleLR scheduler
max_lr = 1e-4
total_steps = len(train_loader) * max_epoch
scheduler = OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps, pct_start=0.3)

# Gradient clipping value
max_norm = 5.0


autoencoder.eval()
while cur_epoch < max_epoch:
    losses = []
    for timeseries, forecast, input_mask in tqdm(train_loader, total=len(train_loader)):
        # Move the data to the GPU
        timeseries = timeseries.float().to(device)
        input_mask = input_mask.to(device)
        forecast = forecast.float().to(device)
        
        with torch.no_grad():
            timeseries = timeseries.permute(0, 2, 1)
            timeseries = autoencoder.encode(timeseries)
            timeseries = timeseries.permute(0, 2, 1)
            forecast = forecast.permute(0, 2, 1)
            forecast = autoencoder.encode(forecast)
            forecast = forecast.permute(0, 2, 1)

        with torch.cuda.amp.autocast():
            # output the forecast

            #timeseries = timeseries.permute(0, 2, 1)
            #timeseries = autoencoder.encode(timeseries)
                # change the shape of the encoded from torch.Size([8, 512, 1]) to torch.Size([8, 1, 512])
            #timeseries = timeseries.permute(0, 2, 1)
            output = model(timeseries, input_mask)
            #output.forecast = output.forecast.permute(0, 2, 1)
            #output.forecast = autoencoder.decode(output.forecast)
            #output.forecast = output.forecast.permute(0, 2, 1)

        loss = criterion(output.forecast, forecast)
        # Scales the loss for mixed precision training
        scaler.scale(loss).backward()

        # Clip gradients
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        losses.append(loss.item())

    losses = np.array(losses)
    average_loss = np.average(losses)
    print(f"Epoch {cur_epoch}: Train loss: {average_loss:.3f}")

    # Step the learning rate scheduler
    scheduler.step()
    cur_epoch += 1

    # Evaluate the model on the test split
    trues, preds, histories, losses = [], [], [], []
    model.eval()
    with torch.no_grad():
        for timeseries, forecast, input_mask in tqdm(test_loader, total=len(test_loader)):
        # Move the data to the GPU
            timeseries = timeseries.float().to(device)
            input_mask = input_mask.to(device)
            forecast = forecast.float().to(device)

            with torch.cuda.amp.autocast():
                # encode the timeseries
                # change the shape of the timeseries from torch.Size([8, 6, 512]) to torch.Size([8, 512, 6])
                timeseries = timeseries.permute(0, 2, 1).to(device)
                #timeseries = (timeseries - mu)/std
                encoded = autoencoder.encode(timeseries)
                # change the shape of the encoded from torch.Size([8, 512, 1]) to torch.Size([8, 1, 512])
                encoded = encoded.permute(0, 2, 1).to(device)

                # output the forecast
                output = model(encoded, input_mask)
                # change the shape of the forecast from torch.Size([8, 1, 192]) to torch.Size([8, 192, 1])
                output.forecast = output.forecast.permute(0, 2, 1).to(device)
                # decode the forecast
                output.forecast = autoencoder.decode(output.forecast)
                #output.forecast = output.forecast * std + mu
                output.forecast = output.forecast.permute(0, 2, 1).to(device)

            loss = criterion(output.forecast, forecast)
            losses.append(loss.item())

            trues.append(forecast.detach().cpu().numpy())
            preds.append(output.forecast.detach().cpu().numpy())
            histories.append(timeseries.detach().cpu().numpy())

    losses = np.array(losses)
    average_loss = np.average(losses)
    model.train()

    trues = np.concatenate(trues, axis=0)
    preds = np.concatenate(preds, axis=0)
    histories = np.concatenate(histories, axis=0)

    metrics = get_forecasting_metrics(y=trues, y_hat=preds, reduction='mean')

    print(f"Epoch {cur_epoch}: Test MSE: {metrics.mse:.3f} | Test MAE: {metrics.mae:.3f}")

100%|██████████| 3276/3276 [02:12<00:00, 24.74it/s]


Epoch 0: Train loss: 0.002


100%|██████████| 102/102 [00:03<00:00, 25.75it/s]


Epoch 1: Test MSE: 12.712 | Test MAE: 2.190


  0%|          | 0/3276 [00:00<?, ?it/s]/home/mohammed/miniconda3/envs/moment/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/mohammed/miniconda3/envs/moment/lib/python3.11/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 3276/3276 [02:20<00:00, 23.37it/s]


...

100%|██████████| 3276/3276 [02:19<00:00, 23.48it/s]


Epoch 9: Train loss: 0.001


100%|██████████| 102/102 [00:03<00:00, 25.80it/s]

Epoch 10: Test MSE: 5.422 | Test MAE: 1.612


: 